In [1]:
import jovian
import opendatasets as od
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
#import pyarrow.feather as feather
import os
import folium
import dask.dataframe as dd
%matplotlib inline

In [2]:
df=pd.read_csv('C:\\Users\\admin\\Downloads\\Parking.csv')

C:\Users\admin\AppData\Local\Temp\ipykernel_28404\3256347001.py:1: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('C:\\Users\\admin\\Downloads\\Parking.csv')


In [3]:
violation_defs_and_fines = pd.read_csv('https://data.cityofnewyork.us/api/views/ncbg-6agr/rows.csv?accessType=DOWNLOAD')
violation_defs_and_fines

,CODE,DEFINITION,Manhattan 96th St. & below,All Other Areas
0,1,FAILURE TO DISPLAY BUS PERMIT,515,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,515,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,515,515
3,4,BUS PARKING IN LOWER MANHATTAN,115,115
4,5,BUS LANE VIOLATION,250,250
...,...,...,...,...
92,93,REMOVE/REPLACE FLAT TIRE,65,65
93,96,RAILROAD CROSSING,95,95
94,97,VACANT LOT,65,45
95,98,OBSTRUCTING DRIVEWAY,95,95


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8198207 entries, 0 to 8198206
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Unnamed: 0           int64  
 1   Registration State   object 
 2   Plate Type           object 
 3   Violation Location   float64
 4   Violation Precinct   int64  
 5   Issuer Precinct      int64  
 6   Issuer Code          int64  
 7   Violation Code       int64  
 8   Meter Number         object 
 9   Violation County     object 
 10  Issuer Squad         object 
 11  Issuing Agency       object 
 12  Issue Date and Time  object 
dtypes: float64(1), int64(5), object(7)
memory usage: 813.1+ MB


In [11]:
cols_to_keep =['Registration State',
               'Plate Type',
               'Violation Location',
               'Violation Precinct', 
               'Issuer Precinct',
               'Issuer Code',
               'Violation Code',
               'Meter Number',
               'Violation County',
               'Issuer Squad',
               'Issuing Agency']

In [12]:
ndf = df[cols_to_keep].copy() # making a copy detaches this from the origional sample_df and raw_df
ndf.head()

,Registration State,Plate Type,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Violation Code,Meter Number,Violation County,Issuer Squad,Issuing Agency
0,NY,PAS,43.0,43,43,972773,67,-,Bronx,0.0,P
1,NY,PAS,14.0,14,968,271057,87,-,Manhattan,0.0,M
2,NY,PAS,14.0,14,968,271057,31,-,NaN,0.0,M
3,99,PAS,108.0,108,968,272834,20,-,NaN,0.0,M
4,PA,PAS,108.0,108,968,272834,40,-,Queens,0.0,M


In [13]:
precinct_loc_df = ndf[['Violation Location', 'Violation Precinct', 'Issuer Precinct']]
precinct_loc_df.describe()

,Violation Location,Violation Precinct,Issuer Precinct
count,4.344860e+06,8.198207e+06,8.198207e+06
mean,5.705748e+01,3.023914e+01,2.912205e+01
std,3.834194e+01,3.987587e+01,5.157755e+01
min,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.900000e+01,0.000000e+00,0.000000e+00
50%,5.200000e+01,6.000000e+00,0.000000e+00
75%,9.400000e+01,6.100000e+01,5.000000e+01
max,9.190000e+02,9.190000e+02,9.980000e+02


In [14]:
%%time
ndf['Issue Date and Time'] = ndf['Issue Date'] + ' ' + ndf['Violation Time']
#ndf['Issue Date and Time'] = ndf['Issue Date and Time'] + 'M'
#ndf['Issue Date and Time'] = pd.to_datetime(df['Issue Date and Time'], format='%m/%d/%Y %I%M%p', errors='coerce')

KeyError: 'Issue Date'

In [15]:
ndf['Issue Date and Time'] = ndf['Issue Date and Time'] + 'M'

KeyError: 'Issue Date and Time'

In [17]:
#ndf['Issue Date and Time'] = pd.to_datetime(ndf['Issue Date and Time'], format='%m/%d/%Y %I%M%p', errors='coerce')

In [ ]:
ndf['Issue Date and Time'].isna().sum()

In [ ]:
ndf['Issue Date and Time'].isna().sum()/ndf.shape[0] * 100

In [ ]:
ndf.drop(['Violation Time', 'Issue Date'], axis=1, inplace=True)
ndf.head(3)

In [ ]:
ndf[ndf['Violation County'].isin(['KINGS', 'QNS', 'BRONX'])]

In [18]:
remap_county_dict = {
    'K' : 'Brooklyn',
    'Q' : 'Queens',
    'NY': 'Manhattan',
    'QN': 'Queens',
    'BK': 'Brooklyn',
    'R' : 'Staten Island',
    'BX': 'Bronx',
    'ST': 'Staten Island',
    'MN': 'Manhattan',
    'KINGS': 'Brooklyn',
    'QNS': 'Queens',
    'BRONX': 'Bronx'
}
ndf['Violation County'] = ndf['Violation County'].map(remap_county_dict).astype('category')
df.head(3)

,Unnamed: 0,Registration State,Plate Type,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Violation Code,Meter Number,Violation County,Issuer Squad,Issuing Agency,Issue Date and Time
0,0,NY,PAS,43.0,43,43,972773,67,-,Bronx,0.0,P,2023-06-09 09:11:00
1,1,NY,PAS,14.0,14,968,271057,87,-,Manhattan,0.0,M,2023-06-30 07:17:00
2,2,NY,PAS,14.0,14,968,271057,31,-,NaN,0.0,M,2023-06-30 08:23:00


In [19]:
ndf['Violation County'].isna().sum()

8198207

In [20]:
ndf['Violation County'].unique()

[NaN]
Categories (0, object): []

In [21]:
county_totals_df = pd.DataFrame(ndf['Violation County'].value_counts())
county_totals_df

,Violation County


In [ ]:
counties = list(ndf['Violation County'][~ndf['Violation County'].isnull()].unique())
for county in counties:
    print('The number of precints in {} data are: {}'.format(county, 
                                              #np.sort(df['Violation Location'][df['Violation County'] == county].unique())))
                                              ndf['Violation Location'][ndf['Violation County'] == county].nunique()))

In [22]:
import json
from urllib.request import urlopen

precinct_geo = 'https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON'

with urlopen(precinct_geo) as response:
    precincts = json.load(response)

In [23]:
type(precincts)


dict

In [29]:
pd.DataFrame([dict])

,0
0,<class 'dict'>


In [25]:
with open("sample.json", "w") as outfile:
    json.dump(precincts, outfile)
    

In [26]:
feature_id_list = []
for index in range(len(precincts['features'])):
    feature_id_list.append(precincts['features'][index]['properties']['precinct'])
np.sort(feature_id_list)

array(['1', '10', '100', '101', '102', '103', '104', '105', '106', '107',
       '108', '109', '110', '111', '112', '113', '114', '115', '120',
       '121', '122', '123', '13', '14', '17', '18', '19', '20', '22',
       '23', '24', '25', '26', '28', '30', '32', '33', '34', '40', '41',
       '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '52',
       '6', '60', '61', '62', '63', '66', '67', '68', '69', '7', '70',
       '71', '72', '73', '75', '76', '77', '78', '79', '81', '83', '84',
       '88', '9', '90', '94'], dtype='<U3')

In [48]:
precincts['features'][index]['properties']['precinct']

'123'

In [46]:
from pandas import json_normalize
df = json_normalize(precincts)

In [47]:
df.to_clipboard(index=False,header=False)

In [44]:
precincts

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'precinct': '1',
    'shape_area': '47285006.2592',
    'shape_leng': '80093.5344285'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-74.04387761573918, 40.6901876758741],
       [-74.0435059601254, 40.68968735963638],
       [-74.0427353382695, 40.6900501909179],
       [-74.04278433380006, 40.69012097669119],
       [-74.04270428426766, 40.690155204644356],
       [-74.04255372037308, 40.68996275928963],
       [-74.04263929371159, 40.6899281759108],
       [-74.0426938081918, 40.68999725910724],
       [-74.04346752310265, 40.68963699010349],
       [-74.04351637245823, 40.68919103323981],
       [-74.04364078627445, 40.68876656007277],
       [-74.04397458556184, 40.68858240705594],
       [-74.04438521777247, 40.68851617790014],
       [-74.04478399040363, 40.6885956601159],
       [-74.04627539003633, 40.68932742539418],
       [-74.04680284898575, 40.689953256266065],
       [-74.

In [49]:
# Extract precinct number
precinct_number = geojson_data['features'][0]['properties']['precinct']

# Extract coordinates
coordinates = geojson_data['features'][0]['geometry']['coordinates']

# Assuming the coordinates are in the form [longitude, latitude]
# You can access them like this
longitude_latitude = coordinates[0][0]  # Assuming the first set of coordinates
longitude, latitude = longitude_latitude

print("Precinct Number:", precinct_number)
print("Longitude:", longitude)
print("Latitude:", latitude)

NameError: name 'geojson_data' is not defined

In [ ]:
df.head()

In [ ]:
train = pd.DataFrame.from_dict(precincts, orient='index')
train.reset_index(level=0, inplace=True)

In [ ]:
train.head()

In [57]:
precinct_numb_df = ndf['Violation Location'][~(ndf['Violation Location'].isnull()) ]. \
                     astype('int32').value_counts().reset_index().rename(columns={'index': 'Violation Location',
                                                                                  'Violation Location': 'counts'})
# the values below need to match the string format of the GeoJson file
precinct_numb_df['Violation Location'] = precinct_numb_df['Violation Location'].astype('str')
# check the precict number is valid and in the geojso list above -- if it is not in the list exclude it
precinct_numb_df = precinct_numb_df[precinct_numb_df['Violation Location'].isin(feature_id_list)]
precinct_numb_df

,Violation Location,counts
0,19,209247
1,6,158075
2,114,157238
3,13,141205
4,14,138000
...,...,...
72,113,15455
73,101,12765
74,100,11494
75,123,6869


In [58]:
precinct_numb_df.to_csv("numb_df")

In [32]:
precinct_dict = {
    'Manhattan': list(range(1,35)),
    'Bronx': list(range(40,53)),
    'Brooklyn': list(range(60,95)),
    'Queens': list(range(100,116)),
    'Staten Island': list(range(120,124))
}

In [33]:
precinct_geo


'https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON'

In [36]:
feature.properties.precinct

NameError: name 'feature' is not defined

In [61]:
import folium

m = folium.Map(location=[40.730610, -73.935242], tiles='CartoDBpositron', zoom_start=10.5)

folium.Choropleth(
        geo_data=precinct_geo,                       # The GeoJSON file from NYC open data
        name="choropleth",
        data=precinct_numb_df,                       # Our grouped DataFrame
        columns=["Violation Location", 'counts'],    # "Violation Location" aligns with feature.properties below
        key_on='feature.properties.precinct',        # Location that aligns with precinct
        fill_color="YlGnBu",
        fill_opacity=0.9,
        line_opacity=0.2,
        legend_name="Tickets issued 2017"
).add_to(m)

folium.LayerControl().add_to(m)

m.save("map.html")  # Save the map to an HTML file


In [62]:
county_df = df['Violation Location'][(df['Violation County'] == county) & 
                                         (df['Violation Location'].isin(precinct_dict[county])) &
                                         (df['Violation Location'].isin(feature_id_list))]

KeyError: 'Violation Location'

In [64]:
rank = pd.DataFrame(ndf[ndf['Violation Location'].isin(feature_id_list)][['Violation Location', 'Violation County']].value_counts()).head(10)
rank

,,0
Violation Location,Violation County,


In [56]:
precint_numb_df.head()

NameError: name 'precint_numb_df' is not defined

In [ ]:
precinct_numb_df.to_csv('C:\\Users\\admin\\Downloads\\precinct_numb_df_df.csv')

In [ ]:
violation_defs_and_fines.columns = ['Violation Code', 'Violation Description', 'Fine (below 96th)', 'Fine']
violation_defs_and_fines.columns

In [ ]:
merged_df = pd.merge(ndf, violation_defs_and_fines, on='Violation Code', how='left')

In [ ]:
merged_df.info()

In [ ]:
fine_df = merged_df[['Violation Location', 'Violation County', 'Violation Description', 'Fine']].copy()

In [ ]:
fine_df.head()

In [ ]:
time_df = ndf[['Issue Date and Time', 'Violation County', 'Violation Location', 'Violation Code']].copy()
time_df 

In [ ]:
time_df['Issue Date and Time'].describe(datetime_is_numeric=True).T

In [ ]:
mask = (time_df['Issue Date and Time'] >= '2016-07-01') & (time_df['Issue Date and Time'] < '2017-07-01')
time_df = time_df.loc[mask].copy()
time_df['Issue Date and Time'].describe(datetime_is_numeric=True)[['min', 'max']].T

In [ ]:
index, county in enumerate(counties):
    county_df = df['Violation Location'][(df['Violation County'] == county) & 
                                         (df['Violation Location'].isin(precinct_dict[county])) &
                                         (df['Violation Location'].isin(feature_id_list))]

In [ ]:
time_df = time_df.set_index('Issue Date and Time', drop=True, append=False, inplace=False, verify_integrity=False).sort_index()

In [ ]:
fine_precinct_df = fine_df.dropna(subset=['Violation Location', 'Violation County']).copy() 
#need to make this a string so the values can be compared witht the valied precincts 
fine_precinct_df['Violation Location'] = fine_precinct_df['Violation Location'].astype('int').astype('str')
# we select out only the valid precinct numbers 
fine_precinct_df = fine_precinct_df[fine_precinct_df['Violation Location'].isin(feature_id_list)] 
fine_precinct_df.sample(5)

In [ ]:
fine_precinct_df.to_csv('C:\\Users\\admin\\Downloads\\fine_precinct_df.csv')


In [ ]:
issuing_agency_df

In [ ]:
import io

web_text = 'A-PORT AUTHORITY, B-TRIBOROUGH BRIDGE AND TUNNEL POLICE, C-CON RAIL, D-DEPARTMENT OF BUSINESS SERVICES, E-BOARD OF ESTIMATE, F-FIRE DEPARTMENT, G-TAXI AND LIMOUSINE COMMISSION, H-HOUSING AUTHORITY, I-STATEN ISLAND RAPID TRANSIT POLICE, J-AMTRAK RAILROAD POLICE, K-PARKS DEPARTMENT, L-LONG ISLAND RAILROAD, M-TRANSIT AUTHORITY, N-NYS PARKS POLICE, O-NYS COURT OFFICERS, P-POLICE DEPARTMENT, Q-DEPARTMENT OF CORRECTION, R-NYC TRANSIT AUTHORITY MANAGERS, S-DEPARTMENT OF SANITATION, T-TRAFFIC, U-PARKING CONTROL UNIT, V-DEPARTMENT OF TRANSPORTATION, W-HEALTH DEPARTMENT POLICE, X-OTHER/UNKNOWN AGENCIES, Y-HEALTH AND HOSPITAL CORP. POLICE, Z-METRO NORTH RAILROAD POLICE, 1-NYS OFFICE OF MENTAL HEALTH POLICE, 2-O M R D D, 3-ROOSEVELT ISLAND SECURITY, 4-SEA GATE ASSOCIATION POLICE, 5-SNUG HARBOR CULTURAL CENTER RANGERS, 6-WATERFRONT COMMISSION OF NY HARBOR, 7-SUNY MARITIME COLLEGE, 9-NYC OFFICE OF THE SHERIFF'
issuing_agency_df = pd.read_csv( io.StringIO(web_text), sep=',', header=None).T.rename(columns={0: 'Agency'})
issuing_agency_df = pd.DataFrame(issuing_agency_df.Agency.str.split('-',1).tolist(),
                                 columns = ['Issuing Agency','Agency Name'])
issuing_agency_df['Issuing Agency'] = issuing_agency_df['Issuing Agency'].str.strip() # Sees to still be whitespace so cnt merge properly
issuing_agency_df

In [ ]:
issuing_agency_df.to_csv('C:\\Users\\admin\\Downloads\\issuing_agency_df.csv')


In [ ]:
issuing_agency_df.to_csv('C:\\Users\\admin\\Downloads\\issuing_agency_df.csv')


In [ ]:
merged_df2 = pd.merge(merged_df, issuing_agency_df, on='Issuing Agency', how='left')

In [ ]:
merged_df2.head()

In [8]:
merged_df2.to_csv('C:\\Users\\admin\\Downloads\\merged_df2.csv')


NameError: name 'merged_df2' is not defined

In [ ]:
temp = pd.DataFrame(merged_df2.groupby(['Agency Name', 'Violation County'])['Plate Type'].count().reset_index())
temp2 = temp.groupby(['Agency Name', 'Violation County'])['Plate Type'].sum().sort_values(ascending=False)
temp3 = pd.DataFrame(temp2).head(19)
temp4 = temp3.reset_index()
temp4

In [ ]:
temp.to_csv('C:\\Users\\admin\\Downloads\\temp.csv')


In [ ]:
ndf.head()

In [ ]:
ndf.to_csv('C:\\Users\\admin\\Downloads\\parking.csv')


In [ ]:
ndf.head()

In [53]:
import json

# Assuming 'data' contains the GeoJSON string
import json
from urllib.request import urlopen

precinct_geo = 'https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON'

with urlopen(precinct_geo) as response:
    geojson_data = json.load(response)

# Parse the GeoJSON data
#geojson_data = json.loads(response)

# Extract precinct number
precinct_number = geojson_data['features'][0]['properties']['precinct']

# Extract coordinates
coordinates = geojson_data['features'][0]['geometry']['coordinates']

# Assuming the coordinates are in the form [longitude, latitude]
# You can access them like this
longitude_latitude = coordinates[0][0]  # Assuming the first set of coordinates
longitude, latitude = longitude_latitude

print("Precinct Number:", precinct_number)
print("Longitude:", longitude)
print("Latitude:", latitude)


ValueError: too many values to unpack (expected 2)

In [54]:
import requests

# Fetch GeoJSON data
url = "https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON"
response = requests.get(url)
data = response.json()

# Extract precinct code, longitude, and latitude
precinct_data = []

for feature in data['features']:
    precinct_code = feature['properties']['precinct']
    coordinates = feature['geometry']['coordinates'][0][0]  # Extracting the first set of coordinates
    longitude, latitude = coordinates
    precinct_data.append({'precinct_code': precinct_code, 'longitude': longitude, 'latitude': latitude})

# Printing the first few entries for verification
for entry in precinct_data[:5]:
    print(entry)


ValueError: too many values to unpack (expected 2)

In [55]:
import requests

# Fetch GeoJSON data
url = "https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON"
response = requests.get(url)
data = response.json()

# Extract precinct code, longitude, and latitude
precinct_data = []

for feature in data['features']:
    precinct_code = feature['properties']['precinct']
    coordinates = feature['geometry']['coordinates'][0]  # Extracting the first set of coordinates
    # Handling MultiPolygon case
    if len(coordinates) == 1:
        longitude, latitude = coordinates[0]
        precinct_data.append({'precinct_code': precinct_code, 'longitude': longitude, 'latitude': latitude})
    else:
        for coord in coordinates:
            longitude, latitude = coord
            precinct_data.append({'precinct_code': precinct_code, 'longitude': longitude, 'latitude': latitude})

# Printing the first few entries for verification
for entry in precinct_data[:5]:
    print(entry)


ValueError: too many values to unpack (expected 2)